In [1]:
import numpy as np
import pandas as pd

def get_pos_list(start, n_step, step_size):
    pos_list = []
    for i in range(n_step):
        pos_list.append(round((start + i * step_size), 4))
    return pos_list

In [22]:
fname = "AM_tomo"
delay = "3"
acq_time = '30'
duplicate = 3

motor1_pos = [96, 30]
motor2_pos = [350, 118, 25]
motor3_pos = get_pos_list(start=92.4, n_step=3, step_size=0.1)
# motor2_pos = get_pos_list(start=5, n_step=3, step_size=0.1)
# motor3_pos = get_pos_list(start=20, n_step=2, step_size=0.1)

motor_dict = {
    "CG1D:Mot:ShortAxis": motor1_pos,
    "CG1D:Mot:LongAxis": motor2_pos,
    "CG1D:Mot:G0:X": motor3_pos,    
}

############################################################################
num_row = 1
num_row_dict = {}
for each_motor in motor_dict.keys():
    _row = len(motor_dict[each_motor])
    num_row = num_row * _row
    num_row_dict[each_motor] = num_row
num_row_dup = num_row * duplicate
loc_to_insert_dict = {}
for each_motor in motor_dict.keys():
    loc_to_insert_dict[each_motor] = np.arange(0, num_row_dup, int(num_row_dup/num_row_dict[each_motor])).tolist()
    
###################################
output_file_name = fname+".csv"
fname_PV = "CG1D:Det:A1:TIFF1:FileName"
camera = "CCD 600"
top_dict = {fname_PV:fname,
            }
if len(motor_dict) != 0 :
    for each_motor in motor_dict.keys():
        top_dict[each_motor] = motor_dict[each_motor]
top_dict["CG1D:Det:A1:AcquireTime"] = acq_time
top_dict["Delay"] = delay
top_dict["Camera"] = camera
content_list = []
for top in top_dict.keys():
    if top in motor_dict.keys():
        content_list.append('')
    elif top == 'Delay':
        content_list.append('')
    else:
        content_list.append(top_dict[top])
scan_dict = {}
for index in range(num_row_dup):
    scan_dict[index] = content_list
scan_df = pd.DataFrame.from_dict(scan_dict, orient='index', columns=top_dict.keys())
for motor in motor_dict.keys():
    motor_dict[motor] = motor_dict[motor] * int(len(loc_to_insert_dict[motor])/len(motor_dict[motor]))
    for n, pos in enumerate(motor_dict[motor]):
        scan_df[motor][loc_to_insert_dict[motor][n]] = pos
_last_motor = list(motor_dict.keys())[-1]
for n, pos in enumerate(motor_dict[_last_motor]):
    scan_df['Delay'][loc_to_insert_dict[_last_motor][n]] = delay
scan_df

,CG1D:Det:A1:TIFF1:FileName,CG1D:Mot:ShortAxis,CG1D:Mot:LongAxis,CG1D:Mot:G0:X,CG1D:Det:A1:AcquireTime,Delay,Camera
0,AM_tomo,96,350,92.4,30,3,CCD 600
1,AM_tomo,,,,30,,CCD 600
2,AM_tomo,,,,30,,CCD 600
3,AM_tomo,,,92.5,30,3,CCD 600
4,AM_tomo,,,,30,,CCD 600
5,AM_tomo,,,,30,,CCD 600
6,AM_tomo,,,92.6,30,3,CCD 600
7,AM_tomo,,,,30,,CCD 600
8,AM_tomo,,,,30,,CCD 600
9,AM_tomo,,118,92.4,30,3,CCD 600


In [23]:
scan_df.to_csv(output_file_name, index=False)